## Overview

The Gemini File API provides a simple way for developers to upload files and use them with the Gemini API in multimodal scenarios. This notebook shows how to use the File API to upload an image and include it in a `GenerateContent` call to the Gemini API.

Note: This API is currently in [private preview](https://github.com/google-gemini/gemini-api-cookbook/blob/main/preview/README.md) and is [only available in certain regions](https://ai.google.dev/available_regions).


## Setup


### Authentication Overview

**Important:** The File API uses API keys for authentication and access. Uploaded files are associated with the API key's cloud project. Unlike other Gemini APIs that use API keys, your API key also grants access data you've uploaded to the File API, so take extra care in keeping your API key secure. For best practices on securing API keys, refer to Google's [documentation](https://support.google.com/googleapi/answer/6310037).

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

### Build a Google API client for the Gemini API

The Gemini API Python SDK does not support the File API [yet](https://github.com/google/generative-ai-python/pull/249). In the meantime, you can build a [Google API Python client](https://github.com/googleapis/google-api-python-client) from the `v1beta` API Discovery Document.

In [ ]:
from googleapiclient.discovery import build
import googleapiclient
import requests

DISCOVERY_URL = f'https://generativelanguage.googleapis.com/$discovery/rest?version=v1beta&key={GOOGLE_API_KEY}';
discovery_doc = requests.get(DISCOVERY_URL).content
genai_service = googleapiclient.discovery.build_from_document(discovery_doc, developerKey=GOOGLE_API_KEY)

## Upload a file to the File API

The File API lets you upload a variety of multi-modal MIME types including images. The File API is only intended as input to generate content and has the following attributes:

* Can only be used with [`model.generateContent`](https://ai.google.dev/api/rest/v1beta/models/generateContent) or [`model.streamGenerateContent`](https://ai.google.dev/api/rest/v1beta/models/streamGenerateContent)
* Automatic file deletion after 2 days
* Maximum 2GB per file, 20GB limit per project
* No downloads allowed

First, click **Files** on the left sidebar, then click **Upload** and upload an image file to Colab.

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

Next, we'll upload that file to the File API. In the form for the code cell below, enter the filename for the file you uploaded and provide an appropriate display name for the file, then run the cell.

In [ ]:
import mimetypes

# Implement the File class expected by the File API
class File:
  def __init__(self, file_path: str, display_name: str = None,
               mimetype: str = None, uri = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    # Detect mimetype if not specified
    self.mimetype = mimetype if mimetype else mimetypes.guess_type(file_path)[0]
    self.uri = uri

  def set_file_uri(self, uri):
    self.uri = uri

# Specify the file to upload
my_filename = "gemini_logo.png" # @param {type:"string"}
my_file_display_name = "Gemini Logo" # @param {type:"string"}

# Create a File
my_file = File(
    file_path=my_filename,
    display_name=my_file_display_name
)

# Upload the file to the File API
response = genai_service.media().upload(
    media_body=my_file.file_path,
    media_mime_type = my_file.mimetype,
    body={"file": {"display_name": my_file.display_name}}).execute()

print(response)

In the `response` you can see the File API generates a random `name` for your uploaded file and a `uri` that you use to reference the file in other Gemini API calls. You should keep track of how your files map to URIs. In this example we save the URI as part of our `File` class, but this could also be stored in a `dict`, a database, or any other strategy that makes sense for your use case.

In [ ]:
my_file.set_file_uri(response["file"]["uri"])

## List Files

`files.list` lets you see all files that have been uploaded to the File API that are associated with the Cloud project your API key belongs to. Only the `name` (and by extension, the `uri`) are unique. Only use the `displayName` to identify files if you manage uniqueness yourself.

In [ ]:
# List files uploaded in the API
response = genai_service.files().list().execute()
print(response)

## Generate Content

After the file has been uploaded, you can make `GenerateContent` requests that reference the File API URI. In this example, `makeGenerateContentRequest()` creates a request that starts with a text `Part` and then has a series of image file `Part`s, but valid `Part`s can come in any order.

In [ ]:
# Make GenerateContent Request
def makeGenerateContentRequest(prompt, files):
  generateContent = {"contents": [{ "parts": [{ "text": prompt }]}]}
  for file in files:
    generateContent["contents"][0]["parts"].append(makeImagePart(file))
  return generateContent

def makeImagePart(file):
  return [
      {"file_data": {"file_uri": file.uri, "mime_type": file.mimetype }}]

prompt = "Describe the image with a creative description" # @param {type:"string"}
model = "models/gemini-1.5-pro-latest" # @param ["models/gemini-1.5-pro-latest", "models/gemini-1.0-pro-vision-latest"]
print(makeGenerateContentRequest(prompt, [my_file]))
response = genai_service.models().generateContent(
    model = model,
    body = makeGenerateContentRequest(prompt, [my_file])).execute()
print(response)

## Delete Files

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [ ]:
# Parse the file name from the URI
resource = my_file.uri.split("/files/")[-1]
response = genai_service.files().delete(name=f"files/{resource}").execute()
print(f'Deleted {my_file.file_path} at URI {my_file.uri}')